![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to boolean data type |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [3]:
import pandas as pd
import numpy as np

#Import the CSV file here:
bank_marketing = pd.read_csv('bank_marketing.csv')

#Change the month values from string to integer
month_to_number = {'jan' : 1,
                  'feb' : 2,
                  'mar' : 3,
                  'apr' : 4,
                  'may' : 5,
                  'jun' : 6,
                  'jul' : 7,
                  'aug' : 8,
                  'sep' : 9,
                  'oct' : 10,
                  'nov' : 11,
                  'dec' : 12}
bank_marketing['month'] = bank_marketing['month'].map(month_to_number)


In [4]:
#Knowing data type of columns
print(bank_marketing.info())
print(bank_marketing.shape)

#Quick overview of dataset
print(bank_marketing.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  int64  
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to boolean data type |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type |

<br>


In [71]:
#Select specific column for Client DataFrame
client_col = bank_marketing[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']]

#Load the specific column to client_df
client = pd.DataFrame(client_col)

#credit_default, mortgage: Convert to boolean datatype
client['credit_default'] = client['credit_default'].astype('bool')
client['mortgage'] = client['mortgage'].astype('bool')

assert client_df['credit_default'].dtype == 'bool'
assert client_df['mortgage'].dtype == 'bool'

#job: Change "." to "_"
client['job'] = client['job'].str.replace("." , "_")
client['job'] = client['job'].replace({"admin_" : "admin"})

#education: Change "." to "_" and "unknown" to np.NaN
client['education'] = client['education'].replace({"." : "_", "unknown" : np.NaN})

print(client.head())


   client_id  age        job  marital    education  credit_default  mortgage
0          0   56  housemaid  married     basic.4y            True      True
1          1   57   services  married  high.school            True      True
2          2   37   services  married  high.school            True      True
3          3   40      admin  married     basic.6y            True      True
4          4   56   services  married  high.school            True      True


## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>


In [72]:
#Select specific column for Campaign DataFrame
campaign_col = bank_marketing[['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome']]

#Load the specific column to campaign_df
campaign = pd.DataFrame(campaign_col)

#Create a year column (value=2022) in bank_marketing dataframe
bank_marketing['year'] = 2022

#Use concat to combine month, day, and year
date_combined = bank_marketing['year'].astype(str) + '-' + bank_marketing['month'].astype(str) + '-' + bank_marketing['day'].astype(str)

#Convert to datetime data type
campaign['last_contact_date'] = pd.to_datetime(date_combined, format='%Y-%m-%d')

#previous_outcome, campaign_outcome : convert to bool
campaign['previous_outcome'] = campaign['previous_outcome'].astype('bool')
campaign['campaign_outcome'] = campaign['campaign_outcome'].astype('bool')

print(campaign.head())

   client_id  number_contacts  ...  campaign_outcome  last_contact_date
0          0                1  ...              True         2022-05-13
1          1                1  ...              True         2022-05-19
2          2                1  ...              True         2022-05-23
3          3                1  ...              True         2022-05-27
4          4                1  ...              True         2022-05-03

[5 rows x 7 columns]


## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [73]:
#Select specific column for Economics DataFrame
economics_col = bank_marketing[['client_id', 'cons_price_idx', 'euribor_three_months']]

#Load the specific column to economics_df
economics = pd.DataFrame(economics_col)

print(economics.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             41188 non-null  int64  
 1   cons_price_idx        41188 non-null  float64
 2   euribor_three_months  41188 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 965.5 KB
None


In [74]:
#Loading the dataframe to csv file

#client_df to client.csv
client.to_csv('client.csv', index=False)

#campaign_df to campaign.csv
campaign.to_csv('campaign.csv', index=False)

#economics_df to economics.csv
economics.to_csv('economics.csv', index=False)
